# Movie Rating Prediction with Python

# Data Collection

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),-2019.0,109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,-2021.0,90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,-2019.0,110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,-2010.0,105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


# Data Preprocessing

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the data with a specified encoding
# Assuming the data is in a CSV file named 'movies.csv'
movies = pd.read_csv('moviesdata.csv', encoding='ISO-8859-1')

# Remove commas from 'Votes' column and convert to float
movies['Votes'] = movies['Votes'].str.replace(',', '').astype(float)

# Remove ' min' from 'Duration' column and convert to float
movies['Duration'] = movies['Duration'].str.replace(' min', '').astype(float)

# Drop rows with any missing values
movies.dropna(inplace=True)

# Encode categorical variables using one-hot encoding
movies = pd.get_dummies(movies, columns=['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3'], drop_first=True)

# Normalize numerical features
scaler = StandardScaler()
movies[['Duration', 'Votes']] = scaler.fit_transform(movies[['Duration', 'Votes']])

# Assuming 'Rating' is the target variable
X = movies.drop('Rating', axis=1)
y = movies['Rating']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')


ValueError: could not convert string to float: '$5.16M'